# Using Gemini in AutoGen with Other LLMs

user_proxy 使用 gemini pro 會報錯

You don't need to handle OpenAI or Google's GenAI packages. AutoGen already handled all of these for you.

You can just create different agents with different backend LLM with assistant agent, and all models/agents are at your fingertip.


## Main Distinctions
- Gemini does not have the "system_message" field (correct me if I am wrong). So, it's instruction following skills are not as strong as GPTs.


Sample OAI_CONFIG_LIST 

```python
[
    {
        "model": "gpt-35-turbo",
        "api_key": "your OpenAI Key goes here",
        "base_url": "https://tnrllmproxy.azurewebsites.net/v1",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "gpt-4-vision-preview",
        "api_key": "your OpenAI Key goes here",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "dalle",
        "api_key": "your OpenAI Key goes here",
        "api_version": "2023-06-01-preview"
    },
    {
        "model": "gemini-pro",
        "api_key": "your Google's GenAI Key goes here",
        "api_type": "google"
    },
    {
        "model": "gemini-pro-vision",
        "api_key": "your Google's GenAI Key goes here",
        "api_type": "google"
    }
]
```

### Before everything starts, install AutoGen with the `gemini` option
```bash
pip install "pyautogen[gemini]~=0.2.0b4"
```


#### Install These Missing Packages Manually if You Encounter Any Errors
```bash
pip install https://github.com/microsoft/autogen/archive/gemini.zip
pip install "google-generativeai" "pydash" "pillow"
```

In [ ]:
# !pip install https://github.com/microsoft/autogen/archive/gemini.zip
# !pip install "google-generativeai" "pydash" "pillow"
import requests
import json
import pdb
import os
import re
import random

from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union
import autogen
from autogen import AssistantAgent, Agent, UserProxyAgent, ConversableAgent


from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.img_utils import get_image_data, _to_pil
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

import chromadb
import PIL
from PIL import Image
from termcolor import colored
import matplotlib.pyplot as plt

In [ ]:
"""
20240219，此處列表更新為5類
gpt35、gpt4、gpt4v、gemini、gemini_vision
"""
config_list_gpt35 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-3.5-turbo-1106", "gpt-3.5-turbo-0613", "gpt-3.5-turbo-16k", "gpt-3.5-turbo-16k-0613"],
    },
)
config_list_gpt4 = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0613", "gpt-4-0314", "gpt-4-1106-preview"],
    },
)
config_list_gpt4v = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4-vision-preview", "dalle"],
    },
)
config_list_gemini = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro"],
    },
)
config_list_gemini_vision = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gemini-pro-vision"],
    },
)

In [ ]:
# 此區塊命令用途，詢問 GPT
"""
這段代碼是使用列表推導式對一系列配置字典中的鍵 "api_version" 進行移除操作。
讓我們來分解這段代碼：
[config.pop("api_version", None) for config in config_list_gpt4]：這行代碼對名為 config_list_gpt4 的列表中的每個配置字典 config 都執行了一個操作，即移除其中的 "api_version" 鍵。如果 "api_version" 鍵不存在於字典中，則不執行任何操作，這就是 pop 方法的作用。列表推導式的結果不會被使用，所以在這個例子中，主要是為了執行 pop 操作。
其餘的行 (config_list_gpt4v, config_list_gpt35, config_list_gemini, config_list_gemini_vision) 也是類似的，只是對應不同的配置列表。
總體來說，這段代碼的作用是從一系列配置字典中移除名為 "api_version" 的鍵，這可能是為了在後續的程式中不再使用該配置參數。
"""
[config.pop("api_version", None) for config in config_list_gpt4]
[config.pop("api_version", None) for config in config_list_gpt4v]
[config.pop("api_version", None) for config in config_list_gpt35]
[config.pop("api_version", None) for config in config_list_gemini]
[config.pop("api_version", None) for config in config_list_gemini_vision]

## Gemini Assitant


In [ ]:
# 定義2個Agent，assistant和user_proxy
# 但不清楚為何跟下面區塊直接設定使用哪個模型不一樣，這裡用 "assistant" 跟 "user_proxy"
# 下面的區塊是用 "gpt-3.5-turbo-1106"、"gemini-pro",
assistant = AssistantAgent(
    "assistant", llm_config={"config_list": config_list_gemini, "seed": 42}, max_consecutive_auto_reply=3
)
# print(assistant.system_message)
# 這個 user_proxy 模型用 "user_proxy" ??? 不懂
# 也沒有 llm_config，但還是可以執行???
# UserProxyAgent 應該只能使用 openAI，其他會報錯，但這個區塊卻又可以????? 不過，整個來說，幾乎都是不行的。
user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": False},
    human_input_mode="NEVER",
    # llm_config={"config_list": config_list_gemini, "seed": 42},
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
)
# 由 user_proxy開始發起對話(initiate_chat)
user_proxy.initiate_chat(assistant, message="請搜尋arXiv上有關醫學領域蛋白質研究最前緣的技術有哪些")  # 這個會報錯
# user_proxy.initiate_chat(assistant, message="Sort the array with Bubble Sort: [4, 1, 5, 2]")     # 這個卻可以 ????

## Agent Collaboration and Interactions



In [ ]:
# 定義2個 agent(gpt、 gemini)，其中 gpt 是用 gpt-3.5-turbo-1106，gemini 是用 gemini-pro。
gpt = AssistantAgent(
    # "GPT-4",
    "gpt-3.5-turbo-1106",
    system_message="""You should ask weird, tricky, and concise questions.
Ask the next question based on (by evolving) the previous one. You should always reply in traditional chinese.""",
    llm_config={"config_list": config_list_gpt35, "seed": 42},
    max_consecutive_auto_reply=3,
)

# gemini 可以用，但很容易安全審查不通過導致無法對話後，就一段時間不回應
gemini = AssistantAgent(
    "gemini-pro",
    system_message="""Always answer questions within one sentence. You should always reply in traditional chinese.""",
    #                      system_message="answer:",
    llm_config={"config_list": config_list_gemini, "seed": 42},
    max_consecutive_auto_reply=4,
)
# 由 gpt 對 gemini 先發起對話，"變形金剛購買汽車保險或健康保險嗎？'
gpt.initiate_chat(gemini, message="變形金剛應該購買汽車保險或健康保險？")

Let's switch position. Now, Gemini is the question raiser. 

This time, Gemini could not follow the system instruction well or evolve questions, because the Gemini does not handle system messages similar to GPTs.

In [ ]:
gpt = AssistantAgent(
    "gpt-3.5-turbo-1106",
    system_message="""Always answer questions within one sentence. You should always reply in traditional chinese.""",
    llm_config={"config_list": config_list_gpt35, "seed": 42},
    max_consecutive_auto_reply=3,
)
# 這裡的 gemini 可以使用
gemini = AssistantAgent(
    "gemini-pro",
    system_message="""You should ask weird, tricky, and concise questions.
Ask the next question based on (by evolving) the previous one. You should always reply in traditional chinese.""",
    llm_config={"config_list": config_list_gemini, "seed": 42},
    max_consecutive_auto_reply=4,
)

gemini.initiate_chat(gpt, message="Should Spider Man invest in 401K?")

## Gemini RAG

Here we will be exploring RAG with Gemini. Note that Gemini will raise a 500 error if a message is an empty string. To prevent this, we set the `default_auto_reply` to `Reply plaintext TERMINATE to exit.` for the `ragproxyagent`.

In [ ]:


# 1. create an RetrieveAssistantAgent instance named "assistant"
assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config={
        "timeout": 600,
        "cache_seed": 42,
        "config_list": config_list_gemini,
    },
)

# 2. create the RetrieveUserProxyAgent instance named "ragproxyagent"
ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent",
    human_input_mode="NEVER",
    default_auto_reply="Reply plaintext TERMINATE to exit.",  # Gemini will raise 500 error if the response is empty.
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": [
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
            "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Research.md",
            os.path.join(os.path.abspath(""), "..", "website", "docs"),
        ],
        "custom_text_types": ["mdx"],
        "chunk_token_size": 2000,
        "model": config_list_gemini[0]["model"],
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "embedding_model": "all-mpnet-base-v2",
        "get_or_create": True,  # set to False if you don't want to reuse an existing collection, but you'll need to remove the collection manually
    },
    code_execution_config=False,  # set to False if you don't want to execute the code
)

code_problem = "How can I use FLAML to perform a classification task and use spark to do parallel training. Train 60 seconds and force cancel jobs if time limit is reached."
ragproxyagent.initiate_chat(
    assistant, problem=code_problem, search_string="spark"
)  # search_string is used as an extra filter for the embeddings search, in this case, we only want to search documents that contain "spark".

## Gemini Multimodal

You can create multimodal agent for Gemini the same way as the GPT-4V and LLaVA.


Note that the Gemini-pro-vision does not support chat yet. So, we only use the last message in the prompt for multi-turn chat. The behavior might be strange compared to GPT-4V and LLaVA models.

Here, we ask a question about 
![](https://github.com/microsoft/autogen/blob/main/website/static/img/chat_example.png?raw=true)

In [ ]:
image_agent = MultimodalConversableAgent(
    "Gemini Vision", llm_config={"config_list": config_list_gemini_vision, "seed": 42}, max_consecutive_auto_reply=1
)

user_proxy = UserProxyAgent("user_proxy", human_input_mode="NEVER", max_consecutive_auto_reply=0)

# user_proxy.initiate_chat(image_agent,
#                          message="""What's the breed of this dog?
# <img https://th.bing.com/th/id/R.422068ce8af4e15b0634fe2540adea7a?rik=y4OcXBE%2fqutDOw&pid=ImgRaw&r=0>.""")

user_proxy.initiate_chat(
    image_agent,
    message="""What is this image about?
<img https://github.com/microsoft/autogen/blob/main/website/static/img/chat_example.png?raw=true>.""",
)

## GroupChat with Gemini Agents

In [ ]:
agent1 = AssistantAgent(
    "Gemini-agent",
    llm_config={"config_list": config_list_gemini, "seed": 42},
    max_consecutive_auto_reply=3,
    system_message="Answer questions about Google.",
    description="I am good at answering questions about Google and Research papers.",
)

agent2 = AssistantAgent(
    "GPT-agent",
    llm_config={"config_list": config_list_gpt4, "seed": 42},
    max_consecutive_auto_reply=3,
    description="I am good at writing code.",
)

user_proxy = UserProxyAgent(
    "user_proxy",
    code_execution_config={"work_dir": "coding", "use_docker": False},
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0
    or content_str(x.get("content")) == "",
    description="I stands for user, and can run code.",
)

groupchat = autogen.GroupChat(agents=[agent1, agent2, user_proxy], messages=[], max_round=12)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={"config_list": config_list_gemini, "seed": 42})

In [ ]:
# user_proxy.initiate_chat(manager, message="Show me the release year of famous Google products.")
user_proxy.send("Show me the release year of famous Google products in a table.", recipient=manager, request_reply=True)

In [ ]:
user_proxy.send(
    "Plot the products and years in scatter plot and save to `graph.png`", recipient=manager, request_reply=True
)

In [ ]:

img = Image.open("coding/graph.png")
img

## A Larger Example of Group Chat

In [ ]:
coder = AssistantAgent(
    name="Coder",
    llm_config={"config_list": config_list_gemini, "seed": 42},
    max_consecutive_auto_reply=10,
    description="I am good at writing code",
)

pm = AssistantAgent(
    name="Product_manager",
    system_message="Creative in software product ideas.",
    llm_config={"config_list": config_list_gemini, "seed": 42},
    max_consecutive_auto_reply=10,
    description="I am good at design products and software.",
)

user_proxy = UserProxyAgent(
    name="User_proxy",
    code_execution_config={"last_n_messages": 20, "work_dir": "coding", "use_docker": False},
    human_input_mode="TERMINATE",
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
    description="I stands for user, and can run code.",
)

groupchat = autogen.GroupChat(agents=[user_proxy, coder, pm], messages=[], max_round=12)
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config={"config_list": config_list_gemini, "seed": 42},
    is_termination_msg=lambda x: content_str(x.get("content")).find("TERMINATE") >= 0,
)
user_proxy.initiate_chat(
    manager,
    message="""Design and implement a multimodal product for people with vision disabilities.
The pipeline will take an image and run Gemini model to describe:
1. what objects are in the image, and
2. where these objects are located.""",
)



## GPT  v.s. Gemini Arena

Do you remember AutoGen can ask LLMs to play chess. Here, we create an arena to allow GPT and Gemini fight together.